In [ ]:
# install huggingface and datasets
!pip install -q datasets transformers
!pip install torch
!pip install pandas
!pip install pyarrow

     |████████████████████████████████| 346 kB 4.1 MB/s 
     |████████████████████████████████| 4.2 MB 68.0 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 140 kB 64.9 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 212 kB 65.1 MB/s 
     |████████████████████████████████| 1.1 MB 53.0 MB/s 
     |████████████████████████████████| 596 kB 56.4 MB/s 
     |████████████████████████████████| 127 kB 69.6 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 80.1 MB/s 
     |████████████████████████████████| 144 kB 56.6 MB/s 
     |████████████████████████████████| 112 kB 75.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 r

In [ ]:
from datasets import load_dataset, ClassLabel
from transformers import BertForTokenClassification, BertTokenizer, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn import metrics


import pandas as pd
import datasets
from datasets.features import ClassLabel

In [ ]:
torch.cuda.is_available()
device = torch.device("cuda")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/e_ML4NLP2/v2.1/'

In [ ]:
train_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/de/HIPE-2022-v2.1-newseye-train-de.tsv'
dev_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/de/HIPE-2022-v2.1-newseye-dev-de.tsv'
test_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/de/HIPE-2022-v2.1-newseye-test_allmasked-de.tsv'
#model = 'dbmdz/flair-clef-hipe-german-base' #not work
#model = 'dbmdz/bert-base-german-europeana-uncased' # not work
#model = 'dbmdz/flair-historic-ner-onb' # not work
#model = 'bert-base-german-dbmdz-uncased' #work
#model = 'xlm-mlm-ende-1024' #multilanguage model engl-de #not try yet
model = "bert-base-german-cased" #work
#model = 'dbmdz/bert-base-german-europeana-cased'

In [ ]:
# import dataset from cloned git repo
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', skip_blank_lines=False, engine='python', quoting=3)
    # error_bad_lines=False, 
    return df


tsv_train = load_dataset(train_path)
tsv_dev = load_dataset(dev_path)
tsv_test = load_dataset(test_path)

In [ ]:
def simple_preprocess(dataframe):
    # Add end_of_document token in df
    dataframe = dataframe.dropna(subset=['TOKEN'])

    # Filter out metadata rows beginning with #
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('#')]
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('\t')]

    #transforming nan var from Float to string to use in (***)
    dataframe.MISC = dataframe.MISC.fillna('')

    return dataframe

In [ ]:
tsv_train = simple_preprocess(tsv_train)
tsv_dev = simple_preprocess(tsv_dev)
tsv_test = simple_preprocess(tsv_test)
#tsv_train = tsv_train.reset_index()
tsv_dev.head(100)

In [ ]:
label_set = tsv_train['NE-COARSE-LIT'].unique()
# define the label mapping for NER
label_list = label_set.tolist()
label_list.append('_')
label_num = len(label_list)

labels = ClassLabel(num_classes=label_num, names=label_list)

labels

In [ ]:
def create_huggingface_file(dataframe):
    import pyarrow as pa
    import re
    from datasets import Dataset

    #creating dataset in json
    hug_out = []
    idx = 0 
    items = {'id': idx,'words':[ ], 'ner': [ ]}
    hug_out.append(items)
    for index, row in dataframe.iterrows():
        if  not re.search(r'EndOfSentence', row['MISC']):
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
        else:
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
            idx += 1
            items = {'id': idx,'words':[ ], 'ner': [ ]}
            hug_out.append(items)
    #filter hug_out out, delete items which has len(words) > 380
    #hug_out = filter(lambda x: len(x['words']) < 380, hug_out)
    #json to df
    hug_out = pd.DataFrame(hug_out)

    # delete all sentences that are too long
    #hug_out = hug_out[hug_out['words'].map(len) < 512] #why does not work? QA

    ### convert to Huggingface dataset
    hug_out = Dataset(pa.Table.from_pandas(hug_out))

    return hug_out

In [ ]:
train = create_huggingface_file(tsv_train)
val = create_huggingface_file(tsv_dev)
test = create_huggingface_file(tsv_test)

#look at training data
for i in range(10):
  print(train[i])

print(len(train))
print(len(val))
print(len(test))

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model)

In [ ]:
def tokenize_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]
    tokenized_ner = []

    for sent_words, sent_nes in zip(tokenized_words, batch["ner"]):
        tokenized_ner.append(
            [labels.str2int("O")]  # BOS symbol
            + [
                ne
                for subwords, ne in zip(sent_words, sent_nes)
                for _ in range(len(subwords))
            ]
            + [labels.str2int("O")]  # EOS symbol
        )

    # Padding with "O"
    tokenized["labels"] = [(ner + [labels.str2int("O")] * (100 - len(ner)))[:100] for ner in tokenized_ner]
    tokenized["subwords"] = tokenized_words
    return tokenized

def tokenize_nolabel_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]

    tokenized["subwords"] = tokenized_words
    return tokenized

#tiny_train_tokenized = tiny_train.map(tokenize_func, batched=True, batch_size=50)
#tiny_test_tokenized = tiny_test.map(tokenize_nolabel_func, batched=True, batch_size=50)

train_tokenized = train.map(tokenize_func, batched=True, batch_size=50)
val_tokenized = val.map(tokenize_func, batched=True, batch_size=50)
test_tokenized = test.map(tokenize_nolabel_func, batched=True, batch_size=50)


In [ ]:
training_args = TrainingArguments(
    output_dir = "content/drive/MyDrive/e_ML4NLP2/"
)

In [ ]:
model = BertForTokenClassification.from_pretrained(model, num_labels=labels.num_classes)
#model = AutoModel.from_pretrained(model_name)
trainer = Trainer(model=model, args=training_args, train_dataset=train_tokenized)
trainer.train()

In [ ]:
predictions = trainer.predict(test_tokenized)

In [ ]:
outfile = open("/content/drive/MyDrive/e_ML4NLP2/teamname_bundle4_de_0.tsv", "w")
outfile.write("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tNE-FINE-LIT\tNE-FINE-METO\tNE-FINE-COMP\tNE-NESTED\tNEL-LIT\tNEL-METO\tMISC")

for pred, tinput in zip(predictions.predictions, test_tokenized):
    pred_word_labels = []

    try:
        sent_len = tinput["input_ids"].index(0)
    except ValueError:
        sent_len = len(tinput["input_ids"])
    # Start at 1 to skip pred for cls token
    i = 1

    for word in tinput['subwords']:
        
        word_score = np.zeros_like(pred[0])
        eos_reached = False
        try:
            for subword in word:
                word_score += pred[i]
                i += 1
            label = labels.int2str(int(np.argmax(word_score)))
            pred_word_labels.append(label)
        except IndexError:
            pred_word_labels.append("O")

    for token, label in zip(tinput['words'], pred_word_labels):
        outfile.write("\n" + f"{token}\t{label}\t" + ('\t'.join(['_'] * 8)))

outfile.close()

In [ ]:
accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    print(logits.shape)
    print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }

In [ ]:
Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
).evaluate()